# Joining NSI data with MOD-IV Data

In [2]:
import os
os.environ['USE_PYGEOS']='0'
import geopandas
import pandas as pd
import numpy as np

In [3]:
# Import data
df_nsi = pd.read_csv('nsi_table.csv')
gdf_nsi = geopandas.GeoDataFrame(
    df_nsi, geometry=geopandas.points_from_xy(df_nsi.Longitude, df_nsi.Latitude, crs="EPSG:4326")
)
gdf_parcels_tmp = geopandas.read_file("parcels and mod4 atlantic county.geojson")

In [ ]:
# Removing duplicates

res=gdf_parcels_tmp.geometry.duplicated(keep=False)
idx = np.where(res)[0]
print("{} parcels, among {} have duplicate foot print polygon shape:".format(len(idx),len(gdf_parcels_tmp)))
count = 0
for i in idx:
    count += 1
    print("count={} i={} {} {}".format(count, i, gdf_parcels_tmp.iloc[i].geometry.centroid,gdf_parcels_tmp.iloc[i].PROP_LOC))

keep_i = [82778 , 110740 , 123408] # choose the ones to keep

counts = len(idx)
gdf_parcels = gdf_parcels_tmp.copy()
for i in range(counts-1,-1,-1):
    if idx[i] not in keep_i:
        gdf_parcels.drop(idx[i], axis=0, inplace=True)

In [9]:
res=gdf_parcels.geometry.duplicated(keep=False)
idx = np.where(res)[0]
print(idx)

[]


# Join to geodata

marge parcels into nsi

In [14]:
gdf_nsi_joined_tmp = geopandas.sjoin(left_df=gdf_nsi,right_df=gdf_parcels, how='left', predicate='covered_by')

In [ ]:
print("nsi={}".format(len(gdf_nsi)))
print("parcels={}".format(len(gdf_parcels)))
print("nsi_joined={}".format(len(gdf_nsi_joined_tmp)))
print("redundant={}".format(len(gdf_nsi_joined_tmp)-len(gdf_nsi)))

nsi=101611
parcels=127236
nsi_joined=101619
redundant=8


In [19]:
#finding nonunique building ids
res=gdf_nsi_joined_tmp.fid.duplicated(keep=False)
idx = np.where(res)[0]
idx

array([13545, 13546, 13548, 13549, 13550, 13551, 25285, 25286, 25287,
       25288, 25289, 25290, 25687, 25688, 25790, 25791], dtype=int64)

In [20]:
fid_array = np.array(gdf_nsi_joined_tmp.fid)
n_unique = len(np.unique(fid_array[idx]))

In [21]:
# Is each parcel unique?
print("{} buildings, among {} have more than one parcel mapped:".format(n_unique,len(gdf_nsi_joined_tmp)))
count = 0
for i in idx:
    count += 1
    print("count={} i={} {} {} {}".format(count, i, fid_array[i], gdf_nsi_joined_tmp.iloc[i].geometry,gdf_nsi_joined_tmp.iloc[i].ST_ADDRESS,gdf_nsi_joined_tmp.iloc[i].PROP_LOC))    

8 buildings, among 101619 have more than one parcel mapped:
count=1 i=13545 761493 POINT (-74.46262067 39.346158516) 112 S WEYMOUTH AVE
count=2 i=13546 761493 POINT (-74.46262067 39.346158516) 4501 ATLANTIC AVE
count=3 i=13548 1234677 POINT (-74.4626003 39.346190385) 112 S WEYMOUTH AVE
count=4 i=13549 1234677 POINT (-74.4626003 39.346190385) 4501 ATLANTIC AVE
count=5 i=13550 1234676 POINT (-74.4626003 39.346190385) 112 S WEYMOUTH AVE
count=6 i=13551 1234676 POINT (-74.4626003 39.346190385) 4501 ATLANTIC AVE
count=7 i=25285 1231558 POINT (-74.4605191 39.351478797) 151 N ANNAPOLIS AVE #8
count=8 i=25286 1231558 POINT (-74.4605191 39.351478797) 151 N ANNAPOLIS AVE
count=9 i=25287 1231559 POINT (-74.4605191 39.351478797) 151 N ANNAPOLIS AVE #8
count=10 i=25288 1231559 POINT (-74.4605191 39.351478797) 151 N ANNAPOLIS AVE
count=11 i=25289 716662 POINT (-74.46051679 39.351482236) 151 N ANNAPOLIS AVE #8
count=12 i=25290 716662 POINT (-74.46051679 39.351482236) 151 N ANNAPOLIS AVE
count=13 i=25

In [22]:
#keep_i = [75188  , 81260  , 120066 , 120068 , 120268 , 120270 , 120327, 120439 ] # choose the ones to keep
keep_i = [13546   ,  13549 , 13551 ,25286 ,25288 ,25290 , 25688  , 25791  ] # choose the ones to keep
#(-74.46262067 39.346158516) 4501 ATLANTIC AVE
# (-74.4626003 39.346190385) 4501 ATLANTIC AVE
# (-74.45871428 39.353330523) 3820 SOUTH BLVD

gdf_nsi_joined = gdf_nsi_joined_tmp.copy().reset_index()
counts = len(idx)
for i in range(counts-1,-1,-1):
    if idx[i] not in keep_i:
        gdf_nsi_joined.drop(idx[i], axis=0, inplace=True)


In [23]:
print("nsi={}".format(len(gdf_nsi)))
print("parcels={}".format(len(gdf_parcels)))
print("nsi_joined={}".format(len(gdf_nsi_joined)))
print("redundant={}".format(len(gdf_nsi_joined)-len(gdf_nsi)))

nsi=101611
parcels=127236
nsi_joined=101611
redundant=0


In [24]:
# import shapely.ops as so
# import matplotlib.pyplot as plt

# pidx = 0

# r1=gdf_parcels.loc[int(gdf_nsi_joined.iloc[idx[pidx]].index_right)].geometry
# r2=gdf_parcels.loc[int(gdf_nsi_joined.iloc[idx[pidx+1]].index_right)].geometry
# r3=gdf_nsi_joined.iloc[idx[pidx]].geometry
# r4=gdf_nsi.iloc[idx[pidx]].geometry

# ax = geopandas.GeoSeries(r2).plot()
# geopandas.GeoSeries(r4).plot(ax=ax, color="red")

# inspect the final table

In [25]:
missing_N = gdf_nsi_joined.index_right.isna().sum()
N =len(gdf_nsi_joined)
missing_perc = missing_N/N
print("missing perc: {} %".format(missing_perc*100))
print("{} among {} missing".format(missing_N, N))

missing perc: 4.9492673037368 %
5029 among 101611 missing


In [37]:
# nonunique_p_N = sum(gdf_nsi_joined.PAMS_PIN.duplicated(keep=False))
nonunique_p_N = sum((1-gdf_nsi_joined.PAMS_PIN.isna())*gdf_nsi_joined.PAMS_PIN.duplicated(keep=False))
nonunique_perc = nonunique_p_N/(N-missing_N)
print("nonunique perc: {} %".format(nonunique_perc*100))
print("{} among {} nonunique".format(nonunique_p_N, N-missing_N))


nonunique perc: 16.71015303058541 %
16139 among 96582 nonunique


In [38]:
nonunique_p_N = sum(gdf_nsi_joined.PAMS_PIN.duplicated(keep=False))
# nonunique_p_N = sum((1-gdf_nsi_joined.PAMS_PIN.isna())*gdf_nsi_joined.PAMS_PIN.duplicated(keep=False))
nonunique_perc = nonunique_p_N/(N-missing_N)
print("nonunique perc: {} %".format(nonunique_perc*100))
print("{} among {} nonunique".format(nonunique_p_N, N-missing_N))


nonunique perc: 21.91712741504628 %
21168 among 96582 nonunique


In [35]:
(gdf_nsi_joined.PAMS_PIN.isna())

0         False
1         False
2         False
3         False
4         False
          ...  
101614    False
101615    False
101616    False
101617    False
101618    False
Name: PAMS_PIN, Length: 101611, dtype: bool

In [27]:
#Final list of columns
for col in gdf_nsi_joined.columns:
    print(col)

index
Unnamed: 0
fid
shape
fd_id
bid
cbfips
st_damcat
occtype
bldgtype
num_story
sqft
found_type
found_ht
med_yr_blt
val_struct
val_cont
val_vehic
ftprntid
ftprntsrc
source
students
pop2amu65
pop2amo65
pop2pmu65
pop2pmo65
o65disable
u65disable
firmzone
grnd_elv_m
ground_elv
Latitude
Longitude
geometry
index_right
OBJECTID
PAMS_PIN
PCL_MUN
PCLBLOCK
PCLLOT
PCLQCODE
PCLLASTUPD
PIN_NODUP
GIS_PIN
CD_CODE
PROP_CLASS
COUNTY
MUN_NAME
PROP_LOC
OWNER_NAME
ST_ADDRESS
CITY_STATE
ZIP_CODE
LAND_VAL
IMPRVT_VAL
NET_VALUE
LAST_YR_TX
BLDG_DESC
LAND_DESC
CALC_ACRE
ADD_LOTS1
ADD_LOTS2
FAC_NAME
PROP_USE
BLDG_CLASS
DEED_BOOK
DEED_PAGE
DEED_DATE
YR_CONSTR
SALES_CODE
SALE_PRICE
DWELL
COMM_DWELL
OLD_PROPID
ZIP5
ZIP_PLUS4
PCL_PBDATE
PCL_GUID
SHAPE_Length
SHAPE_Area


In [39]:
#Chose subset of columns
mylist = ['fid', 'st_damcat', 'occtype', 'bldgtype','num_story','sqft','found_type','found_ht','med_yr_blt','val_struct','val_cont','val_vehic','source','students','pop2amu65','pop2amo65','pop2pmu65','pop2pmo65','o65disable','u65disable','firmzone','grnd_elv_m','ground_elv','Latitude','Longitude','PROP_CLASS','COUNTY','PROP_LOC','CITY_STATE','PROP_USE','BLDG_CLASS','YR_CONSTR']
gdf_nsi_joined_simpler=gdf_nsi_joined[mylist]

In [40]:
gdf_nsi_joined.to_file("./nsi_njdep.geojson", driver="GeoJSON")  

In [41]:
gdf_nsi_joined_simpler.to_csv("./nsi_njdep_simplified.csv")  